In [92]:
import os
import time
import numpy as np
start = time.time()
Slength = 0
hashList=[]
bandNum = 50
FunNumber = 300
trainPath = "./2000train/*.txt"
testPath = "./testDoc/*.txt"
def generateHashFun():
    '''300 hash functions'''
    x = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,
         113,127,131,137,139,149,151,157,163,167,173,179,181,191,193,197,199,211,223,227,229,233,
         239,241,251,257,263,269,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,367,
         373,379,383,389,397,401,409,419,421,431,433,439,443,449,457,461,463,467,479,487,491,499,
         503,509,521,523,541,547,557,563,569,571,577,587,593,599,601,607,613,617,619,631,641,643,
         647,653,659,661,673,677,683,691,701,709,719,727,733,739,743,751,757,761,769,773,787,797,
         809,811,821,823,827,829,839,853,857,859,863,877,881,883,887,907,911,919,929,937,941,947,
         953,967,971,977,983,991,997,1009,1013,1019,1021,1031,1033,1039,1049,1051,1061,1063,1069,
         1087,1091,1093,1097,1103,1109,1117,1123,1129,1151,1153,1163,1171,1181,1187,1193,1201,1213,
         1217,1223,1229,1231,1237,1249,1259,1277,1279,1283,1289,1291,1297,1301,1303,1307,1319,1321,
         1327,1361,1367,1373,1381,1399,1409,1423,1427,1429,1433,1439,1447,1451,1453,1459,1471,1481,
         1483,1487,1489,1493,1499,1511,1523,1531,1543,1549,1553,1559,1567,1571,1579,1583,1597,1601,
         1607,1609,1613,1619,1621,1627,1637,1657,1663,1667,1669,1693,1697,1699,1709,1721,1723,1733,
         1741,1747,1753,1759,1777,1783,1787,1789,1801,1811,1823,1831,1847,1861,1867,1871,1873,1877,
         1879,1889,1901,1907,1913,1931,1933,1949,1951,1973,1979,1987]
    for i in x:
        hashList.append(_hf(i))
def _hf(multy):
    return lambda x: ((multy % Slength) * (x%Slength) +1) % Slength

def caculateKeyID(string):
    '''
    Caculate key words' ID by its value
    '''
    sumID = 0
    for i in range(0,len(string),2):
        sumID = (ord(string[i]) - 64) + i*26
    return sumID
def getKeyWords(wordList):
    '''
    If word List is ['A','B','C','D','E','F','G'],
    Return Value is ['ABC','BCD','CDE','DEF','EFG']
    '''
    arr = []
    for i in range(len(wordList)-3):
        arr.append("".join(wordList[i:i+3]))
    return arr

def getHashValues(item):
    '''
    item is like (("wordA wordB wordC",[columnX1,columnX2,columnX2]),rowID)
    With a List of Hash Functions, caculate hash values by rowID
    return a list of hash values
    '''
    arr = []
    for i in range(FunNumber):
        arr.append(((i,hashList[i](item[1])),item[0][1]))
    return arr

def parallelRead(path):
    '''
    Read directory in parallel, Return a RDD List of item 
    who is like (("wordA wordB wordC",[columnX1,columnX2,columnX2]),rowID)
    Where "wordA wordB wordC" appear continuously and columnX is the document ID it appears
    '''
    rdd = sc.wholeTextFiles(path,4).map(lambda x: (x[0],x[1].split(" "))).map(lambda x: (x[0],getKeyWords(x[1]))).zipWithUniqueId()
    fileDict = rdd.map(lambda x:(x[0][0],x[1]))
    dataRDD = rdd.map(lambda x:(x[1],x[0][1])).flatMapValues(lambda x: x).map(lambda x: (x[1],x[0])).groupByKey().mapValues(list).map(lambda x:((x[0],x[1]),caculateKeyID(x[0])))
    return dataRDD,fileDict
def getSignature(X):
    '''
    Return a RDD--Signature Matrix with shape ((rowID, columnID),hashValue)
    '''
    X1 = X.map(lambda x:getHashValues(x)).flatMap(lambda x: x).flatMapValues(lambda x: x)
    return X1.map(lambda x: ((x[0][0],x[1]),x[0][1])).groupByKey().mapValues(lambda x: min(list(x)))

def equalBucket(List):
    '''
    List is ((bandID, columnID),[((bandID, columnID),value),((bandID, columnID),value),((bandID, columnID),value)])
    use a hash function to get bucket ID
    '''
    tp = ''
    for i in range(0,len(List),2):
        tp+=str(List[i][1])
    return tp
X,fileDict = parallelRead(trainPath)
Slength = X.count()
generateHashFun()
Signature = getSignature(X)
'''
bucket's item like ((bandRowID,backetValue),[columnID_1,columnID_2...])
'''
BucketTable = Signature.groupBy(lambda x: (x[0][1],x[0][0] % bandNum)).mapValues(list).map(lambda x: ((x[0][1],equalBucket(x[1])),x[0][0])).groupByKey().mapValues(list)
testDocs,testName = parallelRead(testPath)
testSignature = getSignature(testDocs)
testBucket = testSignature.groupBy(lambda x: (x[0][1],x[0][0] % bandNum)).mapValues(list).map(lambda x: ((x[0][1],equalBucket(x[1])),x[0][0])).groupByKey().mapValues(list)
def getPossibleDoc():
    '''
    columnID is ID in signature Matrix, which is actually documentID
    return RDD's item is like (columnID_AIM,[columnID_X1,columnID_X2,columnID_X3,...,columnID_Xn]) 
    columnID_AIM is the document we want to find its similar docs
    columnID_X is possible similar docs in same bucket
    '''
    def Filter(x):
        if x[0]==None:
            return ''
        return (x[1],x[0])
    T = testBucket.leftOuterJoin(BucketTable).map(lambda x: (x[1][1],x[1][0])).flatMapValues(lambda x: x).filter(lambda x:x[0]!=None).map(lambda x:(x[1],x[0]))
    return T.groupByKey().mapValues(list).map(lambda x: (x[0],list(set(reduce(lambda a,b:a+b, x[1])))))
Arrs = getPossibleDoc().collect()
test_ID = 0
Source_ID = Arrs[0][1]
tempFileName = fileDict.filter(lambda x:x[1] in Source_ID).map(lambda x: ((x[1],0),x[0]))
T2 = testSignature.filter(lambda x: x[0][1] == test_ID).map(lambda x: ((x[0][0],x[1]),x[0][1]))
T1 = Signature.filter(lambda x: x[0][1] in Source_ID).map(lambda x: ((x[0][0],x[1]),x[0][1]))
print (time.time() - start)

147.256175995


In [93]:
start = time.time()
jacSim = T1.join(T2).map(lambda x: (x[1],x[0])).groupByKey().mapValues(len).map(lambda x: (x[0],x[1]/float(2 * FunNumber - x[1])))
FindFileName = [(x, tuple(map(list, y))) for x, y in sorted(list(jacSim.cogroup(tempFileName).collect()))]
print (time.time() - start)

36.3618099689


In [95]:
start = time.time()
newjacSim = Signature.map(lambda x: ((x[0][0],x[1]),x[0][1])).join(testSignature.map(lambda x: ((x[0][0],x[1]),x[0][1]))).map(lambda x: (x[1],x[0])).groupByKey().mapValues(len).map(lambda x: (x[0],x[1]/float(2 * FunNumber - x[1])))
newFindFileName = [(x, tuple(map(list, y))) for x, y in sorted(list(newjacSim.cogroup(tempFileName).collect()))]
print (time.time() - start)

37.2761211395
